In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import re

**Downloading files directly from Dropbox to Colab**

In [2]:
!wget 'https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0'

--2024-06-22 13:00:09--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/bnabda9cee2yuyi6wesmv/stacksample.zip?rlkey=sucdc1cm2m7fnwtzhykv24g8z&dl=0 [following]
--2024-06-22 13:00:10--  https://www.dropbox.com/scl/fi/bnabda9cee2yuyi6wesmv/stacksample.zip?rlkey=sucdc1cm2m7fnwtzhykv24g8z&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc080c605917d1c73cf916f31d7c.dl.dropboxusercontent.com/cd/0/inline/CVVKHa9tiKT4VQ8rpOgadyULSFqtLDd3LVNS0CMky3LVMi3537YsXaMChxRFaW7yZ3xR35UgIOFldIFplJrgptjsEWKsz6uQcpG5YrOX_6l9nLMfBP-UYNEJz54XtY-Yk70/file# [following]
--2024-06-22 13:00:10--  https://uc080c605917d1c73cf916f31d7c.dl.dropboxusercontent.com/cd/0/in

**Installing Unzip**

In [3]:
!apt install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


**Unzipping contents**

In [4]:
!unzip 'stacksample.zip'

Archive:  stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [5]:
ques = pd.read_csv("Questions.csv", encoding='latin-1')

In [6]:
ans = pd.read_csv("Answers.csv", encoding='latin-1')

In [7]:
tags = pd.read_csv("Tags.csv", encoding='latin-1')

In [8]:
ques.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body'],
      dtype='object')

In [9]:
ques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [10]:
ans.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], dtype='object')

In [11]:
tags.columns

Index(['Id', 'Tag'], dtype='object')

In [12]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [13]:
tags['Tag'].value_counts().head(10)

Tag
javascript    124155
java          115212
c#            101186
php            98808
android        90659
jquery         78542
python         64601
html           58976
c++            47591
ios            47009
Name: count, dtype: int64

In [14]:
top10tags = tags['Tag'].value_counts().head(10).index.tolist()

In [15]:
##unique_id = ques['Id'].unique().tolist()

**Creating new tags dataframe with only top 10 tags**

In [16]:
tag_counts = tags['Tag'].value_counts()
tags_new = tags[tags['Tag'].isin(tag_counts[tag_counts > 47000].index)]

In [17]:
tags.shape[0]

3750994

In [18]:
tags_new.shape[0]

826739

In [19]:
dummy_cols = pd.get_dummies(tags_new['Tag'])

In [20]:
df_concat = pd.concat([tags_new['Id'], dummy_cols], axis=1)

In [27]:
df_concat.loc[df_concat['Id'] == 4009250]

,Id,android,c#,c++,html,ios,java,javascript,jquery,php,python
271474,4009250,False,True,False,False,False,False,False,False,False,False
271475,4009250,False,False,False,False,False,False,False,False,True,False
271476,4009250,False,False,True,False,False,False,False,False,False,False
271477,4009250,False,False,False,False,False,False,False,False,False,True
271478,4009250,False,False,False,True,False,False,False,False,False,False


In [28]:
df_pivot = df_concat.pivot_table(index='Id', aggfunc='sum')

In [32]:
df_pivot

,android,c#,c++,html,ios,java,javascript,jquery,php,python
Id,,,,,,,,,,
260,0,1,0,0,0,0,0,0,0,0
330,0,0,1,0,0,0,0,0,0,0
650,0,1,0,0,0,0,0,0,0,0
930,0,1,0,0,0,0,0,0,0,0
1010,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
40143150,0,0,0,0,0,0,1,0,0,0
40143190,0,0,0,0,0,0,0,0,0,1
40143210,0,0,0,0,0,0,0,0,1,0


In [33]:
tags_new.loc[tags_new['Id'] == 4009250]

,Id,Tag
271474,4009250,c#
271475,4009250,php
271476,4009250,c++
271477,4009250,python
271478,4009250,html


In [34]:
tags_new['Id'].value_counts()

Id
4009250     5
30798790    5
30289880    5
23484760    5
35318730    5
           ..
16405420    1
16405630    1
16405650    1
16405690    1
40143360    1
Name: count, Length: 706336, dtype: int64

**Creating dummies of top 10 tags for each Ids**

In [35]:
df_final = df_pivot.reset_index()

In [31]:
df_final.loc[df_final['Id'] == 4009250]

,Id,android,c#,c++,html,ios,java,javascript,jquery,php,python
43176,4009250,0,1,1,1,0,0,0,0,1,1


In [36]:
df_final.shape[0]

706336

In [37]:
df_final.columns

Index(['Id', 'android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'],
      dtype='object')

**Preparing Questions data with tags information**

In [38]:
train_data = pd.merge(ques, df_final, on='Id', how='inner')

In [39]:
train_data.shape[0]

706336

In [40]:
train_data.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body', 'android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'],
      dtype='object')

In [41]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706336 entries, 0 to 706335
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            706336 non-null  int64  
 1   OwnerUserId   697889 non-null  float64
 2   CreationDate  706336 non-null  object 
 3   ClosedDate    39134 non-null   object 
 4   Score         706336 non-null  int64  
 5   Title         706336 non-null  object 
 6   Body          706336 non-null  object 
 7   android       706336 non-null  int64  
 8   c#            706336 non-null  int64  
 9   c++           706336 non-null  int64  
 10  html          706336 non-null  int64  
 11  ios           706336 non-null  int64  
 12  java          706336 non-null  int64  
 13  javascript    706336 non-null  int64  
 14  jquery        706336 non-null  int64  
 15  php           706336 non-null  int64  
 16  python        706336 non-null  int64  
dtypes: float64(1), int64(12), object(4)
memory usage

In [42]:
train_data.drop(['OwnerUserId', 'CreationDate', 'ClosedDate', 'Score'], axis = 1, inplace = True)

In [43]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706336 entries, 0 to 706335
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Id          706336 non-null  int64 
 1   Title       706336 non-null  object
 2   Body        706336 non-null  object
 3   android     706336 non-null  int64 
 4   c#          706336 non-null  int64 
 5   c++         706336 non-null  int64 
 6   html        706336 non-null  int64 
 7   ios         706336 non-null  int64 
 8   java        706336 non-null  int64 
 9   javascript  706336 non-null  int64 
 10  jquery      706336 non-null  int64 
 11  php         706336 non-null  int64 
 12  python      706336 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 70.1+ MB


**Changing display width**

In [44]:
pd.set_option('display.max_colwidth', None)

In [45]:
train_data['Body'].head(1)

0    <p>I have a little game written in C#. It uses a database as back-end. It's \na <a href="http://en.wikipedia.org/wiki/Collectible_card_game">trading card game</a>, and I wanted to implement the function of the cards as a script.</p>\n\n<p>What I mean is that I essentially have an interface, <code>ICard</code>, which a card class implements (<code>public class Card056 : ICard</code>) and which contains function that are called by the game.</p>\n\n<p>Now, to make the thing maintainable/moddable, I would like to have the class for each card as source code in the database and essentially compile it on first use. So when I have to add/change a card, I'll just add it to the database and tell my application to refresh, without needing any assembly deployment (especially since we would be talking about 1 assembly per card which means hundreds of assemblies).</p>\n\n<p>Is that possible? Register a class from a source file and then instantiate it, etc.</p>\n\n<pre><code>ICard Cards[current]

In [46]:
train_data['Body'] = train_data['Body'].astype(str)

**Removing HTML Tags from the Body**

In [47]:
train_data['Body'] = train_data['Body'].apply(lambda x: re.sub(r"<[^>]*>", "", x))

In [48]:
ans['Body'] = ans['Body'].apply(lambda x: re.sub(r"<[^>]*>", "", x))

**Removing escape sequences or whitespace characters**

In [49]:
train_data['Body'] = train_data['Body'].apply(lambda x: re.sub(r'[\n\t\f\r\v]', "", x))

In [50]:
ans['Body'] = ans['Body'].apply(lambda x: re.sub(r'[\n\t\f\r\v]', "", x))

In [51]:
train_data['Body'].head(1)

0    I have a little game written in C#. It uses a database as back-end. It's a trading card game, and I wanted to implement the function of the cards as a script.What I mean is that I essentially have an interface, ICard, which a card class implements (public class Card056 : ICard) and which contains function that are called by the game.Now, to make the thing maintainable/moddable, I would like to have the class for each card as source code in the database and essentially compile it on first use. So when I have to add/change a card, I'll just add it to the database and tell my application to refresh, without needing any assembly deployment (especially since we would be talking about 1 assembly per card which means hundreds of assemblies).Is that possible? Register a class from a source file and then instantiate it, etc.ICard Cards[current] = new MyGame.CardLibrary.Card056();Cards[current].OnEnterPlay(ref currentGameState);The language is C#, but extra bonus if it's possible to write t

**Removing double quotes and single quotes**

In [53]:
train_data['Body'] = train_data['Body'].apply(lambda x: re.sub(r"(\"|\')", "", x))

**Splitting into train test data**

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
x_train, x_test = train_test_split(train_data, test_size=0.2, shuffle=True, stratify=None)

In [56]:
train_data.shape[0]

706336

In [57]:
x_train.shape[0]

565068

In [58]:
x_test.shape[0]

141268

**Importing required libraries**

In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

In [47]:
'''text_data = train_data['Body']
target_data = train_data[['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript', 'jquery', 'php', 'python']]'''

In [62]:
text_data = x_train['Body']
target_data = x_train[['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript', 'jquery', 'php', 'python']]

In [63]:
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(text_data)

In [64]:
model = MultiOutputClassifier(estimator=LogisticRegression())
model.fit(features, target_data)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

MultiOutputClassifier(estimator=LogisticRegression())

In [60]:
x_test.sample(4)

,Id,Title,Body,android,c#,c++,html,ios,java,javascript,jquery,php,python
193697,13205240,How do I trigger javascript to execute on document windown resize (without refreshing the page)?,"Im having a problem getting javascript to execute whenever I resize the browser window. I am using the following javascript:$(document).ready(function(){ var heightNum = $(#work-details-info).height() + 39; if (($(document).width() &gt; 767) &amp;&amp; ($(document).width() &lt; 900)) { $(#work-details-left-cntr).css(height, (heightNum)); }});Its calculating the height of a specific div. Then it is adding 39px to that height, and it sets the new height to another div. This works perfectly whenever I resize the page and then press refresh/reload. However, I want this to trigger without having to refresh/reload. How do I do that?",0,0,0,0,0,0,1,0,0,0
243282,16072110,Android: DialogFragment and universal xml,"I am trying to take control of the database in android with Dialog Fragments. I mean, for example to add a new record I will click a button and pop-up fragment appears asking me for the specific fields. I click ok which fires the method in my hosting activity. That part works. However, I also want to have other operations like delete, update, search record ect. Is there a way to have a universal code for fragment but then assign different xml according to different database operations?I am looking for the most efficient way around my problem. Thanks!",1,0,0,0,0,0,0,0,0,0
372284,23171800,StyleCop plugin for overriding / implementing summary XML,"I am exceptionally anal when it comes to StyleCop. I like to ensure that my class libraries are well laid out, maintainable, and appropriately documented.Say for example I create an interface:public interface ICustomImpl{ /// &lt;summary&gt; /// Say hello! /// &lt;/summary&gt; void SayHello();}and then I implement my interfacepublic class Hello : ICustomImpl{ public void SayHello() { //impl }}StyleCop will now complain that I have not added a summary tag to my method implementation, however I have a summary tag on the interface.Are there any known plugins that will automatically copy summary tags where available?Note: I would like this to work for interface implementations and method overrides.",0,1,0,0,0,0,0,0,0,0
571779,33705750,Android Bundle Iterator to Put and Remove Entries,"I am seeking a method to iterate over the entries contained in a Bundle object. Currently, I am incorrectly using a For Each approach:for (String key : bundle.keySet()) { if (videourl.equals(key.toLowerCase()) || priority.equals(key.toLowerCase())) { // Save value to bundle using lowercase key value extras.putString(key.toLowerCase(), (String) extras.get(key)); // Remove original version of this bundle entry extras.remove(key); }}The problem with the approach shown above is that it risks arbitrary, non-deterministic behavior at an undetermined time in the future.Ive seen Iterator implementations for Map objects (e.g. http://stackoverflow.com/a/1884916/3957979):for(Iterator&lt;Map.Entry&lt;String, String&gt;&gt; it = map.entrySet().iterator(); it.hasNext(); ) { Map.Entry&lt;String, String&gt; entry = it.next(); if(entry.getKey().equals(test)) { it.remove(); }}However, I dont have direct access to the Bundles entries so I cant use the same approach.Please let me know if Iterators can be used with Bundle objects or please suggest a different approach.",1,0,0,0,0,1,0,0,0,0


**Taking one of the sample Body from x_test**

In [75]:
new_text = ["I am exceptionally anal when it comes to StyleCop. I like to ensure that my class libraries are well laid out, maintainable, and appropriately documented.Say for example I create an interface:public interface ICustomImpl{ /// &lt;summary&gt; /// Say hello! /// &lt;/summary&gt; void SayHello();}and then I implement my interfacepublic class Hello : ICustomImpl{ public void SayHello() { //impl }}StyleCop will now complain that I have not added a summary tag to my method implementation, however I have a summary tag on the interface.Are there any known plugins that will automatically copy summary tags where available?Note: I would like this to work for interface implementations and method overrides."]

In [76]:
new_features = vectorizer.transform(new_text)

In [77]:
predicted_tags = model.predict(new_features)

# Print the predicted tags
print(predicted_tags)

[[0 1 0 0 0 0 0 0 0 0]]


In [78]:
cols= ['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript', 'jquery', 'php', 'python']

**Flattening the prediction**

In [79]:
predicted_tags_flattened = predicted_tags.flatten()
test = pd.DataFrame({'column_name': cols, 'predicted_value': predicted_tags_flattened})
print(test)

  column_name  predicted_value
0     android                0
1          c#                1
2         c++                0
3        html                0
4         ios                0
5        java                0
6  javascript                0
7      jquery                0
8         php                0
9      python                0
